In [4]:
%pip install --upgrade pip 
%pip install --upgrade transformers datasets[audio] accelerate

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
os.environ["PATH"] += os.pathsep + r"/opt/homebrew/Cellar/ffmpeg/8.0_1/bin/ffmpeg" # 자신이 설치한 위치로 경로 수정

In [1]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
# from datasets import load_dataset

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
    return_timestamps=True,   # 청크별로 타임스탬프 반환
    chunk_length_s=10,  # 입력 오디오 10초씩 나누기r
    stride_length_s=2,  # 2초씩 겹치도록 청크 나누기
) 

# dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
# sample = dataset[0]["audio"]
sample = "C:/Users/gunwo/anaconda3/envs/TEST/llm/싼기타_비싼기타.mp3"

result = pipe(sample)
# print(result["text"])

print(result)


c:\Users\gunwo\anaconda3\envs\TEST\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cpu
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/con

{'text': ' 지금부터 저랑 그 역할극을 합시다 역할극을 스탠딩 코미디 스타일로 할 건데 토론을 하면서 자연스럽게 대화하는 형식으로 하면서 코미디를 진행해봅시다. 그래서 좀 재밌고 자연스럽고 유머러스하게 저랑 대화를 하시면 돼요 자연스럽게 그리고 주제는 쌍기타로 전기기타를 시작하는 게 좋으냐 아니면 비싼 기타로 전기기타를 시작하는 게 좋으냐 이거를 입장을 나눠가지고 저랑 토론해보면 좋을 것 같아요. 두 중에 어떤 역할 맡으실래요? 좋습니다. 그럼 제가 싼 기타로 시작하는 게 좋다는 입장을 맡아볼게요. 그럼 성형님은 비싼 기타로 시작하는 게 좋다는 입장이시죠? 네 맞아요. 준비 되셨나요? 네 됐어요. 시작하시죠. 좋아요. 먼저 3기타로 시작하는게 좋은 이유를 말씀드리겠습니다. 초보자일때는 실수도 많고 기타에 익숙해지는 과정이 필요하니까 비싼 기타보다는 부담없이 연습할 수 있는 싼 기타가 좋다고 봅니다. 무엇보다 실력이 향상되면 그때 좋은 기타로 업그레이드 하는 것도 나쁘지 않죠? 성형님은 어떻게 생각하세요? 아, 저는 지금 말에 어폐가 있다고 생각해요. 왜냐하면 어차피 지금 비싼 기타로 나중에 살 생각을 하는 거잖아요. 그럼 어차피 비싼 기타로 사고 싼 기타로 사게 되는데 그런 이중 이중 지출을 하지 말고 처음부터 비싼 기타로 가면은 이중지출을 하지 말고 처음부터 비싼 기타로 가면 이중지출을 막을 수가 있습니다. 그런데 비싼 기타로 시작하면 혹시라도 흠집이 나거나 실수할 때 부담이 더 크지 않을까요? 초보 때는 실습을 많이 하니까 더 크지 않을까요? 초보 때는 실습을 많이 하니까 쌍기타로 마음 편히 연습할 수 있는게 더 좋다고 생각해요. 그리고 기타 현지에 익숙해지면 그때 비싼 기타를 사는게 더 의미있지 않을까요? 아니 근데 어차피 비싼 기타를 살 건데 싼 기타를 뭐하러 또 삽니까? 그리고 기타 실력이랑 흠집이랑은 아무 상관이 없어요 기타를 망치로 치는 것도 아니라 손가락으로 치는 건데 기타를 싼 거 산다고 달라지나요? 하하, 기타를 망치로 치진 않지만 그

In [2]:
# chunks를 CSV 파일로 저장
start_end_text = []

for chunk in result["chunks"]:
    start = chunk["timestamp"][0]
    end = chunk["timestamp"][1]
    text = chunk["text"]
    start_end_text.append([start, end, text])

import pandas as pd
df = pd.DataFrame(start_end_text, columns=["start", "end", "text"])
df.to_csv("싼기타_비싼기타_final.csv", index=False, sep="|")
display(df)


,start,end,text
0,1.00,11.00,지금부터 저랑 그 역할극을 합시다 역할극을 스탠딩 코미디 스타일로 할 건데 토론을...
1,12.00,17.00,그래서 좀 재밌고 자연스럽고 유머러스하게 저랑 대화를 하시면 돼요 자연스럽게
2,17.00,21.46,그리고 주제는 쌍기타로 전기기타를 시작하는 게 좋으냐
3,21.46,24.36,아니면 비싼 기타로 전기기타를 시작하는 게 좋으냐
4,24.36,26.00,이거를 입장을 나눠가지고
...,...,...,...
119,414.54,417.00,계속해서 이하고 나누고 싶으시면 편하게 말씀해주세요.
120,417.00,418.00,아니요.
121,418.00,420.00,화나셨는데 굳이 더 할 필요 없죠.
122,420.00,423.00,그만 잊지 마시죠.
